#Master file to create Corpus for Kubernetes

Author:  Doug Williams, Copyright 2014, 2015

Last Updated: 2/4/2015

Can either rebuild corpus from scratch, or apply updates incrementally.   Since some of the cata is downloaded from the web, there are a set of intermediate corpus that are also maintained.  These are later joined to produce the finl result

Corpus:
- kubernetes_commits.jsonz


#Installation Instructions:

######Need latest version of GitPython
- pip install -I GitPython==0.3.2.RC1

In [1]:
import pprint as pp
import sys
sys.path.append('../dev')

from jp_load_dump import pdump, pload, jdump, jload

from Git_Extract_Join import build_git_commits, load_git_commits
from Git_Extract_Join import build_joined_LP_Gerrit_git, load_combined_commits
from Git_Extract_Join import build_all_blame, load_all_blame

### Parameters

In [2]:
PROJECT = 'kubernetes'
REPO_NAME ='/Users/doug/SW_Dev/kubernetes'
CACHEDIR = './cache/'
UPDATE_CORPUS = True
BUILD_CORPUS = False

### Update base data for Corpus

In [3]:
if BUILD_CORPUS or UPDATE_CORPUS:
    print
    print'building Git data'
    build_git_commits(project=PROJECT, repo_name=REPO_NAME)


building Git data
. . . . . . . . . . 1000 . . . . . . . . . . 2000 . . . . . . . . . . 3000 . . . . . . . . . . 4000 . . . . . . . . . . 5000 . . . . . .
total commits: 5605
Augment Git data with patch info
known patches: 0
requested patches: 0
new patches to be fetched: 0
Augment Git data with ordering info

Augment Git data with lines-of-code changed
. . . . . . . . . . 1000 . . . . . . . . . . 2000 . . . . . . . . . . 3000 . . . . . . . . . . 4000 . . . . . . . . . . 5000 . . . . . .


### Get Launchpad, Gerrit and Git Data

In [4]:
print 'loading Git commit data'
commits = load_git_commits(PROJECT)

loading Git commit data
returning singleton
total git_commits: 5605
bug fix commits: 1
commits with change_id: 0
bug fix with change_id: 0


###### Sanity check data

In [5]:
total_bugs = 0
total_changes = 0
total_both = 0
for x in commits.values():
    if 'bug' in x:
        total_bugs += 1
        
print 'commits:', len(commits)
print 'bugs:', total_bugs

sample_change = [x for x in commits.values() if 'change_id' in x]
print 'changes:', len(sample_change)
sample_both = [x for x in commits.values() if 'change_id' in x and 'bug' in x]
print 'bug fix changes:', len(sample_both)

print
pp.pprint(commits.values()[0:20])

commits: 5605
bugs: 1
changes: 0
bug fix changes: 0

[{u'author': u'<git.Actor "Joe Beda <joe@bedafamily.com>">',
  u'author_order': 18,
  u'cid': u'6df5afb88e9c5a0e17aee67fd82d9e003a2b6068',
  u'committer': u'<git.Actor "Joe Beda <joe@bedafamily.com>">',
  u'date': 1411491506,
  u'file_order': {u'hack/test-integration.sh': 31},
  u'file_order_for_author': {u'hack/test-integration.sh': 2},
  u'files': [u'hack/test-integration.sh'],
  u'loc_add': 29,
  u'loc_change': 29,
  u'loc_detail': {u'hack/test-integration.sh': {u'add': 29, u'changes': 29}},
  u'msg': u'Merge pull request #1412 from filbranden/test_integration1\n\nCleanup hack/test-integration.sh',
  u'order': 2240,
  u'parents': [u'7743f33359e8fc054f82b0ecf884a8a9d909b372',
               u'30db3e281de44eb0fc355db23b32c12cf7f5c922']},
 {u'author': u'<git.Actor "Dawn Chen <dawnchen@google.com>">',
  u'author_order': 67,
  u'cid': u'e91f9087e9deead55dd0dcd087a87831adc5f4a1',
  u'committer': u'<git.Actor "Dawn Chen <dawnchen@google.

# Pull Requests

###Check for merge pull request'

Sample: Merge pull request #289 from thockin/cleanups

Handling: substitute source repo base name (e.g.: thockin) as 'author' of commit

In [6]:
total_pulls = 0
for x in commits.values():
    if 'msg' in x and 'merge pull request #' in x['msg'].lower():
        total_pulls += 1
        
print 'commits:', len(commits)
print 'pulls:', total_pulls

commits: 5605
pulls: 2251


###Check for handling of phrase 'merge branch'

Conclusion:  ignore

In [7]:
total_merge_branch = 0
for x in commits.values():
    if 'msg' in x and x['msg'].lower().startswith('merge branch'):
        total_merge_branch += 1
        print x['msg']
        print '--------------------'
        
print 'commits:', len(commits)
print 'merge_branch:', total_merge_branch

Merge branch 'master' of github.com:jdef/kubernetes

--------------------
Merge branch 'rename-to-kube' of https://github.com/eparis/kubernetes into eparis-rename-to-kube

* 'rename-to-kube' of https://github.com/eparis/kubernetes:
  rename kube server binaries to kube-

Conflicts:
	docs/salt.md

--------------------
Merge branch 'master' of github.com:GoogleCloudPlatform/kubernetes

--------------------
Merge branch 'eparis-rename-to-kube'

* eparis-rename-to-kube:
  rename kube server binaries to kube-

--------------------
Merge branch 'master' of github.com:GoogleCloudPlatform/kubernetes into configurable-sleep-interval

--------------------
Merge branch 'master' into fix/golint2

Conflicts:
	pkg/api/types.go
	pkg/health/health_check.go
	pkg/kubelet/kubelet.go
	pkg/kubelet/kubelet_server.go
	pkg/kubelet/kubelet_server_test.go
	pkg/kubelet/kubelet_test.go

--------------------
Merge branch 'master' into fix/golint

Conflicts:
	pkg/master/master.go
	pkg/master/pod_cache.go
	pkg/proxy

###Check for handling of phrase 'merge' outside of 'merge pull request' or 'merge branch'

Conclusion:  ignore

In [8]:
print '*** Abnormal commits - messages contain merge ***'
print 
abnormal = 0
for x in commits.values():
    if 'msg' in x and'merge' in x['msg'].lower() and not 'merge pull request #' in x['msg'].lower() and not x['msg'].lower().startswith('merge branch'):
        print  x['msg']
        print '--------------------'   
        abnormal += 1
print 'total abnormal pull requests:', abnormal

*** Abnormal commits - messages contain merge ***

Fix inflight merge conflict by adapting rollingupdate to #2861

ClientConfig changed right as rollingupdate was introduced.

--------------------
Fix localkube due to a bad merge.

--------------------
Revert "Fix inflight merge conflict by adapting rollingupdate to #2861"

--------------------
Update design doc with final PR merge

--------------------
Add name generation and normalize common create flows

Adds `ObjectMeta.GenerateName`, an optional string field that defines
name generation behavior if a Name is not provided.

Adds `pkg/api/rest`, which defines the default Kubernetes API pattern
for creation (and will cover update as well). Will allow registries
and REST objects to be merged by moving logic on api out of those places.

Add `pkg/api/rest/resttest`, which will be the test suite that verifies
a RESTStorage object follows the Kubernetes API conventions and begin
reducing our duplicated tests.

--------------------
Documen

###Check for handling of phrase 'pull' outside of 'merge pull request'

Conclusion:  ignore, pull typically used in a non-merge related context

In [9]:
print
print '*** Abnormal commits - messages contain pull ***'
print 
abnormal = 0
for x in commits.values():
    if 'msg' in x and 'pull' in x['msg'].lower() and not 'merge pull request #' in x['msg'].lower():
        print  x['msg']
        print '--------------------'   
        abnormal += 1
print 'total abnormal pull requests:', abnormal


*** Abnormal commits - messages contain pull ***

Confirm pull of golang:cross as it is huge (1.8G)

--------------------
Add skydns to deps.

godep build fails without this.
Also pulls in github.com/miekg/dns.
rackspace/gophercloud doesn't seem to need kr/text and golang-pretty anymore.

--------------------
If the image with :latest tag specified in Spec, kubelet should try
to pull the latest one even the policy is PullIfNotPresent.

--------------------
If no tag was specified, pull "latest" explicitly.

When no tag is given to Docker pull, it downloads all tags. This could be a
significantly large number of images and in the end we only ever run
:latest.

--------------------
Add the ability to turn off image pulling.

--------------------
Updated guestbook based on suggestions in the pull request

Port of current guestbook's README.md

Fix guestbook pod and service names

Add go boilerplate

Use role label for redis pods

Give service.containerPort a name based value

This gives 

# Bug Fixes

In [10]:
total_bugs = 0
total_fixs = 0
total_issues = 0
total_hashes = 0

for x in commits.values():
    if 'msg' in x and 'bug' in x['msg'].lower():
        total_bugs += 1
    if 'msg' in x and 'fix' in x['msg'].lower():
        total_fixs += 1
    if 'msg' in x and 'issue' in x['msg'].lower():
        total_issues += 1
        
    if 'msg' in x and '#' in x['msg'].lower() and not 'merge pull request #' in x['msg'].lower():
        total_hashes += 1
        
print 'commits:', len(commits)
print 'bugs:', total_bugs
print 'fixes:', total_fixs
print 'issues:', total_issues
print 'messages containing # that are not pull requests:', total_hashes

commits: 5605
bugs: 75
fixes: 1325
issues: 93
messages containing # that are not pull requests: 164


### Sample messages containing fix

In [11]:
total_fixes = 0
for x in commits.values():
    if 'msg' in x and 'fix' in x['msg'].lower():
        total_fixes += 1
        print  x['msg']
        print '--------------------'   
        
    if total_fixes > 10:
        break

Fix bug in minion registrytest.

--------------------
Fix docker start after minion reboot on GCE/debian

Uncomment docker service state as it should no longer be affected by race bug in docker init.d start script.  Also make sure that the docker service state depends on the cbr state.

Fixes #802

--------------------
Merge pull request #2309 from lavalamp/fix3

Refactors of kubectl object printers.
--------------------
Clean up READMEs and broken stuff.

Move all support for clouds that are broken with this change into an `icebox` directory.  We'll move that stuff back out as we fix it up.

--------------------
Fix paths for kubectl

--------------------
Fix a problem with for loops, copy semantics and async routines.

--------------------
Merge pull request #339 from thockin/valid8

Fix wrong json name for Key field
--------------------
Merge pull request #3444 from lavalamp/fix

eliminate possibility of double-calling
--------------------
Merge pull request #89 from jbeda/gcloud-fi

### Messages contining bug

In [12]:
for x in commits.values():
    if 'msg' in x and 'bug' in x['msg'].lower():
        print  x['msg']
        print '--------------------'   

Fix bug in minion registrytest.

--------------------
Fix docker start after minion reboot on GCE/debian

Uncomment docker service state as it should no longer be affected by race bug in docker init.d start script.  Also make sure that the docker service state depends on the cbr state.

Fixes #802

--------------------
Add a better test for service environment variables
Fix bugs.

--------------------
Remove one line of debug code.

--------------------
Add a little bit more logging to a test so that if it times out we have more debugging.

--------------------
Remove debug print in kubectl.sh

Removes a debug print that (it seems to me) was added in erro
as part of PR #3046 / commit c3efef8.

--------------------
Fix subtle bug when proxy constructs outgoing URL.

--------------------
Improve test coverage for the health package and remove mocks

The current tests for the health package utilize a fake HTTP client
for testing health checks and only cover a limited set of test cases.

T

### Messages contining issue

In [13]:
for x in commits.values():
    if 'msg' in x and 'issue' in x['msg'].lower():
        print  x['msg']
        print '--------------------'   

NewMainKubelet validates Kubelet config parameters

Fixes issue #3202.
* Validates SyncFrequency and minimum GC age and propagates error to RunKubelet
* Defaults for Kubelet config and minor cleanup
* cmd Kubelet MinimumGCAge to 1m instead of 0

--------------------
Replace kubelet logging %s--%s--%s with dockertools.BuildDockerName()

Fixes issue #3454

--------------------
Merge pull request #2910 from smarterclayton/small_v1beta3_refactors

Fix internal -> v1beta3 round trip issues
--------------------
Add test timing to TAP output in hack/e2e.go

Found this buried at https://issues.jenkins-ci.org/browse/JENKINS-16647

--------------------
Adding docs for prioritization of issues.

--------------------
Merge pull request #3972 from derekwaynecarr/fix_etcd_vagrnat

Vagrant fixup for etcd 2.0 regression, and other issues
--------------------
GKE: Add ENABLE_CLUSTER_DNS, compute appropriate DNS_SERVER_IP

Fixes https://github.com/GoogleCloudPlatform/kubernetes/issues/3191

------------

### Non merge request messages with '#'

In [14]:
for x in commits.values():        
    if 'msg' in x and '#' in x['msg'].lower() and not 'merge pull request #' in x['msg'].lower():
        print  x['msg']
        print '--------------------'   

Fix docker start after minion reboot on GCE/debian

Uncomment docker service state as it should no longer be affected by race bug in docker init.d start script.  Also make sure that the docker service state depends on the cbr state.

Fixes #802

--------------------
Actually delete files (completes #1807)

--------------------
Update exec health check message logged

Update message logged after performing exec health check per #2708.
--------------------
Do not leak undocumented variables from config-go.sh

Run the snippet that creates the output/go/ tree in a subshell.

Tested:
- Built it with hack/build-go.sh
- Ran release/build-release.sh successfully.
- Moved away the git tree to confirm no regression in PR #1073.
- Sourced hack/config-go.sh in the shell, confirmed no variable or
  function other than the expected ones leaked into the environment.
- Used `git grep` to confirm the no longer exported variables were
  not in use by any script other than config-go.sh.

Signed-off-by: F

### Compute Diff and Blame